In [1]:
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, models, datasets, utils
from torchsummary import summary
from PIL import Image
import memory_profiler  # conda install -c anaconda memory_profiler
import matplotlib.pyplot as plt
from torch.utils.data import SubsetRandomSampler
from cnn_utils import weights_init, trainer, cnn_feature_importance, get_test_accuracy, pytorch_confusion_matrix, recall_calculation

In [2]:
import os
os.getcwd()

'C:\\Users\\szyan\\Documents\\Capstone'

In [3]:
#transform

IMAGE_SIZE = 300
BATCH_SIZE = 8

TRAIN_DIR = "data/train/"
VALID_DIR = "data/val/"


train_transforms = transforms.Compose([
     transforms.RandomVerticalFlip(p=0.5),
     transforms.RandomRotation(degrees=20),
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.ToTensor()
])

valid_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.ImageFolder(root=TRAIN_DIR, transform=train_transforms)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataset = torchvision.datasets.ImageFolder(root=VALID_DIR, transform=valid_transforms)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [4]:
inception_v3 = densenet = models.inception_v3(pretrained=True)
inception_v3.fc = nn.Linear(2048, 1)
inception_v3.aux_logits = False

In [5]:
torch.manual_seed(123)
inception_v3.apply(weights_init);

# Move to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
inception_v3.to(device);

Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Initializing weights of a Conv2d layer!
Initializing weights of a batchnorm layer!
Init

In [6]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(inception_v3.parameters())
results = trainer(inception_v3, criterion, optimizer, train_loader, valid_loader, device, epochs=20)

Epoch 1: Train Accuracy: 0.66. Valid Accuracy: 0.56. Valid Recall: 0.60
Epoch 2: Train Accuracy: 0.69. Valid Accuracy: 0.48. Valid Recall: 0.00
Epoch 3: Train Accuracy: 0.71. Valid Accuracy: 0.67. Valid Recall: 0.60
Epoch 4: Train Accuracy: 0.70. Valid Accuracy: 0.74. Valid Recall: 0.75
Epoch 5: Train Accuracy: 0.83. Valid Accuracy: 0.82. Valid Recall: 0.85
Epoch 6: Train Accuracy: 0.79. Valid Accuracy: 0.72. Valid Recall: 0.60
Epoch 7: Train Accuracy: 0.85. Valid Accuracy: 0.76. Valid Recall: 0.90
Epoch 8: Train Accuracy: 0.80. Valid Accuracy: 0.77. Valid Recall: 0.65
Epoch 9: Train Accuracy: 0.82. Valid Accuracy: 0.75. Valid Recall: 0.75
Epoch 10: Train Accuracy: 0.79. Valid Accuracy: 0.80. Valid Recall: 0.90
Epoch 11: Train Accuracy: 0.83. Valid Accuracy: 0.76. Valid Recall: 0.75
Epoch 12: Train Accuracy: 0.84. Valid Accuracy: 0.72. Valid Recall: 0.65
Epoch 13: Train Accuracy: 0.83. Valid Accuracy: 0.75. Valid Recall: 0.90
Epoch 14: Train Accuracy: 0.83. Valid Accuracy: 0.82. Valid 

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
TEST_DIR = "data/test/"
test_dataset = torchvision.datasets.ImageFolder(root=TEST_DIR, transform=valid_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
get_test_accuracy(inception_v3, test_loader)

0.8333333333333334

In [9]:
cmtx = pytorch_confusion_matrix(2,inception_v3, test_loader)
cmtx

,predict negative,predict positive
actual negative,15.0,5.0
actual positive,3.0,18.0


In [11]:
def recall_calculation(confusion_matrix):
    return confusion_matrix.iloc[1,1]/(confusion_matrix.iloc[1,1]+confusion_matrix.iloc[1,0])

In [12]:
recall_calculation(cmtx)

0.85714287